# システム統合３（フレーム＋機械学習言語理解）
それでは、これまでに実装してきたモジュールを統合して音声対話システムを作成してみましょう。

- 仕様
    - 音声認識：Google（ストリーミング、オリジナルVAD）
    - 音声合成：Google
    - 言語理解：機械学習（レストラン案内のスロット値推定）
    - 対話管理：フレーム

In [1]:
# 必要なラブラリを読み込む
# from __future__ import division

# import re
# import sys, os

# import numpy as np
# import math
# import struct

# import pyaudio
# from six.moves import queue

# from pydub import AudioSegment
# from pydub.playback import play

# from google.cloud import speech

# 実装したライブラリを読み込む
from asr_google_streaming_vad import GoogleStreamingASR, MicrophoneStream
from tts_google import GoogleTextToSpeech
from dm_frame import DmFrame
from slu_ml import SluML

C:\Users\inoue\anaconda3\envs\ai-kouza\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# 音声認識クラスのパラメータ
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

# 音声合成の初期化
tts = GoogleTextToSpeech()

# 言語理解の初期化
slu_parser = SluML()

In [6]:
# 対話管理の初期化
dm = DmFrame()

# 初期状態の発話
system_utterance = dm.utterance_start
tts.generate(system_utterance)
print("システム： " + system_utterance)
tts.play()

# 全てのフレームが埋まるまで対話を続ける
while(dm.current_frame_filled == False):

    # 音声認識入力を得る	
    micStream = MicrophoneStream(RATE, CHUNK)
    asrStream = GoogleStreamingASR(RATE, micStream)
    print('<<<please speak>>>')
    result_asr = asrStream.get_asr_result()

    if hasattr(result_asr, 'alternatives') == False:
        print('Invalid ASR input')
        continue

    result_asr_utterance = result_asr.alternatives[0].transcript
    print("ユーザ： " + result_asr_utterance)
    
    # 言語理解
    result_slu = slu_parser.extract_slot_restaurant(result_asr_utterance)
    print(result_slu)
    
    # 対話管理へ入力
    system_utterance = dm.enter(result_slu)
    tts.generate(system_utterance)
    print("システム： " + system_utterance)
    tts.play()
    
    print()


システム： こんにちは。京都レストラン案内です。ご質問をどうぞ。
<<<please speak>>>
音声パワー： 46.8[dB] 途中結果: 出町柳周辺で何か美味しいものが食べたいユーザ： 出町柳周辺で何か美味しいものが食べたい
出町柳	B-place
周辺	I-place
で	O
何	O
か	O
美味しい	O
もの	O
が	O
食べ	O
たい	O
[{'intent': '', 'slot_name': 'place', 'slot_value': '出町柳周辺'}]
システム： ジャンルを指定してください
<<<please speak>>>
音声パワー： 48.7[dB] ユーザ： イタリアンはどうですかか
イタリアン	B-genre
は	O
どう	O
です	O
か	O
[{'intent': '', 'slot_name': 'genre', 'slot_value': 'イタリアン'}]
システム： 地域は出町柳周辺で、ジャンルはイタリアンですね。検索します。
